In [23]:
# Install required libraries
!pip install scikit-fuzzy torch torchvision

# Import libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
import skfuzzy as fuzz
import skfuzzy.membership as mf
from google.colab import files
import matplotlib.pyplot as plt
import seaborn as sns

# 🚀 Upload CSV file in Google Colab
uploaded = files.upload()

# 📂 Read CSV file
file_name = list(uploaded.keys())[0]  # Get the uploaded file name
df = pd.read_csv(file_name, sep=';', decimal=',', encoding='latin1')

# 🧹 Data Preprocessing
df = df.drop(columns=['Unnamed: 15', 'Unnamed: 16'], errors='ignore')  # Remove empty columns
df = df.dropna()  # Drop missing values

# Remove non-numeric columns
df_numeric = df.drop(columns=['Date', 'Time'], errors='ignore')

# Define features and target variable
X = df_numeric.drop(columns=['CO(GT)'], errors='ignore')
y = df_numeric['CO(GT)']

# Split data into Train (60%), Validation (20%), Test (20%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 📊 Standardize Data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 🔹 RBF Network Implementation
class RBFNetwork:
    def __init__(self, num_centers):
        self.num_centers = num_centers  # Number of RBF centers
        self.centers = None
        self.weights = None

    def fit(self, X, y):
        kmeans = KMeans(n_clusters=self.num_centers, random_state=42)
        kmeans.fit(X)
        self.centers = kmeans.cluster_centers_
        Phi = np.exp(-cdist(X, self.centers, 'sqeuclidean'))
        self.weights = np.linalg.pinv(Phi).dot(y)

    def predict(self, X):
        Phi = np.exp(-cdist(X, self.centers, 'sqeuclidean'))
        return Phi.dot(self.weights)

# 🚀 Train RBF Model
num_centers = 10  # Number of cluster centers
rbf_model = RBFNetwork(num_centers=num_centers)
rbf_model.fit(X_train_scaled, y_train)

# Predict using RBF Network
y_train_pred_rbf = rbf_model.predict(X_train_scaled)
y_test_pred_rbf = rbf_model.predict(X_test_scaled)

# Compute evaluation metrics for RBF
rbf_metrics = {
    "Train RMSE": np.sqrt(mean_squared_error(y_train, y_train_pred_rbf)),
    "Test RMSE": np.sqrt(mean_squared_error(y_test, y_test_pred_rbf)),
    "Train MAE": mean_absolute_error(y_train, y_train_pred_rbf),
    "Test MAE": mean_absolute_error(y_test, y_test_pred_rbf),
    "Train R²": r2_score(y_train, y_train_pred_rbf),
    "Test R²": r2_score(y_test, y_test_pred_rbf),
}

# 🔹 ANFIS Model
class ANFIS(nn.Module):
    def __init__(self, input_dim, num_rules):
        super(ANFIS, self).__init__()
        self.input_dim = input_dim
        self.num_rules = num_rules

        # Membership function parameters (Gaussian)
        self.mf_centers = nn.Parameter(torch.randn(num_rules, input_dim))  # Rule centers
        self.mf_widths = nn.Parameter(torch.randn(num_rules, input_dim).abs())  # Rule spreads

        # Rule weights for output layer
        self.rule_weights = nn.Parameter(torch.randn(num_rules, input_dim))

        # Output layer
        self.output_layer = nn.Linear(num_rules, 1)

    def forward(self, x):
        x_expanded = x.unsqueeze(1)  # Expand dimensions for broadcasting
        gauss = torch.exp(-((x_expanded - self.mf_centers) ** 2) / (2 * self.mf_widths ** 2))
        rule_activations = torch.prod(gauss, dim=2)
        rule_activations = rule_activations / (torch.sum(rule_activations, dim=1, keepdim=True) + 1e-6)
        output = self.output_layer(rule_activations)
        return output

# 🚀 Train ANFIS Model
num_rules = 5
anfis_model = ANFIS(input_dim=X_train.shape[1], num_rules=num_rules)

optimizer = optim.Adam(anfis_model.parameters(), lr=0.01)
loss_function = nn.MSELoss()

# Convert data to PyTorch tensors
X_train_torch = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_torch = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test_torch = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_torch = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

num_epochs = 100
for epoch in range(num_epochs):
    optimizer.zero_grad()
    y_train_pred_anfis = anfis_model(X_train_torch)
    loss = loss_function(y_train_pred_anfis, y_train_torch)
    loss.backward()
    optimizer.step()

# Predict using ANFIS Model
y_train_pred_anfis = anfis_model(X_train_torch).detach().numpy().flatten()
y_test_pred_anfis = anfis_model(X_test_torch).detach().numpy().flatten()

# Compute evaluation metrics for ANFIS
anfis_metrics = {
    "Train RMSE": np.sqrt(mean_squared_error(y_train, y_train_pred_anfis)),
    "Test RMSE": np.sqrt(mean_squared_error(y_test, y_test_pred_anfis)),
    "Train MAE": mean_absolute_error(y_train, y_train_pred_anfis),
    "Test MAE": mean_absolute_error(y_test, y_test_pred_anfis),
    "Train R²": r2_score(y_train, y_train_pred_anfis),
    "Test R²": r2_score(y_test, y_test_pred_anfis),
}

# 📌 Display results in Markdown
from IPython.display import display, Markdown

markdown_text = f"""
# 🔥 Model Performance Comparison

| Metric | RBF Train | RBF Test | ANFIS Train | ANFIS Test |
|--------|----------|---------|------------|------------|
| RMSE   | {rbf_metrics["Train RMSE"]:.4f} | {rbf_metrics["Test RMSE"]:.4f} | {anfis_metrics["Train RMSE"]:.4f} | {anfis_metrics["Test RMSE"]:.4f} |
| MAE    | {rbf_metrics["Train MAE"]:.4f} | {rbf_metrics["Test MAE"]:.4f} | {anfis_metrics["Train MAE"]:.4f} | {anfis_metrics["Test MAE"]:.4f} |
| R²     | {rbf_metrics["Train R²"]:.4f} | {rbf_metrics["Test R²"]:.4f} | {anfis_metrics["Train R²"]:.4f} | {anfis_metrics["Test R²"]:.4f} |

**Conclusion:** The model with better test RMSE, MAE, and R² performs better.
"""

display(Markdown(markdown_text))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

Saving AirQualityUCI.csv to AirQualityUCI (3).csv



# 🔥 Model Performance Comparison

| Metric | RBF Train | RBF Test | ANFIS Train | ANFIS Test |
|--------|----------|---------|------------|------------|
| RMSE   | 63.8315 | 58.4548 | 84.7555 | 81.3496 |
| MAE    | 34.1610 | 30.5757 | 39.7031 | 36.9192 |
| R²     | 0.3341 | 0.4044 | -0.1741 | -0.1535 |

**Conclusion:** The model with better test RMSE, MAE, and R² performs better.
